In [ ]:
import numpy as np

class GaussianNB:
    def __init__(self):
        self.classProbabilities = {}
        self.mean = {}
        self.var = {}

    def fit(self, X, y):
        classes = np.unique(y)
        
        for c in classes:
            XClasses = X[y == c]
            self.classProbabilities[c] = len(XClasses) / len(X)

            self.mean[c] = np.mean(XClasses, axis=0)
            self.var[c] = np.var(XClasses, axis=0)

    def predict(self, X):
        predictions = [self.predictSample(x) for x in X]
        return np.array(predictions)

    def predictSample(self, x):
        classProbabilities = {}

        for c in self.classProbabilities:
            probability = np.log(self.classProbabilities[c])
            for i in range(len(x)):
                probability += self.calculateLikelihood(x[i], c, i)

            classProbabilities[c] = probability
        
        return max(classProbabilities, key=classProbabilities.get)

    def calculateLikelihood(self, x_i, c, featureIndex):
        mean = self.mean[c][featureIndex]
        var = self.var[c][featureIndex]

        return -0.5 * np.log(2 * np.pi * var) - 0.5 * ((x_i - mean) ** 2) / var

In [ ]:
import numpy as np

class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y == c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)
            

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = posterior + prior
            posteriors.append(posterior)

        # return class with the highest posterior
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

In [ ]:
import numpy as np

class GaussianNB:
    def __init__(self, var_smoothing=1e-9):
        self.var_smoothing = var_smoothing

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_features = X.shape[1]
        n_classes = len(self.classes_)

        self.theta_ = np.zeros((n_classes, n_features))
        self.var_ = np.zeros((n_classes, n_features))
        self.class_prior_ = np.zeros(n_classes)

        for i, c in enumerate(self.classes_):
            X_c = X[y == c]
            self.theta_[i] = np.mean(X_c, axis=0)
            self.var_[i] = np.var(X_c, axis=0) + self.var_smoothing
            self.class_prior_[i] = X_c.shape[0] / X.shape[0]

    def predict(self, X):
        log_likelihood = []
        for i in range(len(self.classes_)):
            prior = np.log(self.class_prior_[i])
            likelihood = -0.5 * np.sum(np.log(2 * np.pi * self.var_[i]))
            likelihood -= 0.5 * np.sum(((X - self.theta_[i]) ** 2) / self.var_[i], axis=1)
            log_likelihood.append(prior + likelihood)

        return self.classes_[np.argmax(np.array(log_likelihood).T, axis=1)]

In [ ]:
import numpy as np

class GaussianNaiveBayes:
    def __init__(self, var_smoothing=1e-9):
        self.var_smoothing = var_smoothing
        self.class_probs = {}
        self.means = {}
        self.vars = {}

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_samples, n_features = X.shape

        for c in self.classes_:
            X_c = X[y == c]
            self.class_probs[c] = X_c.shape[0] / n_samples
            self.means[c] = np.mean(X_c, axis=0)
            self.vars[c] = np.var(X_c, axis=0) + self.var_smoothing

    def predict(self, X):
        return np.array([self._predict_sample(x) for x in X])

    def _predict_sample(self, x):
        posteriors = {}

        for c in self.classes_:
            prior = np.log(self.class_probs[c])
            likelihood = np.sum(self._calculate_likelihood(x, c))
            posteriors[c] = prior + likelihood

        return max(posteriors, key=posteriors.get)

    def _calculate_likelihood(self, x, c):
        mean = self.means[c]
        var = self.vars[c]
        return -0.5 * np.log(2 * np.pi * var) - 0.5 * ((x - mean) ** 2) / var